# About DataSet

O conjunto de dados "Netflix Movies and TV Shows" é um conjunto de dados tabulares com a lista de todos os filmes e séries da Netflix. Os campos incluem diretor, título, classificação, ano de lançamento, duração e muito mais.

# Análise

1. Distribuição de conteúdo
2. Top 10 países produtores
3. Conteúdos adicionados por ano ao catálogo
4. Principais gêneros dos conteúdos
5. Ator/Atriz com mais participações em conteúdos


# Importação dos dados

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('./dataset/netflix_titles_CLEANED.csv')

# Limpeza dos Dados

In [16]:
df.fillna({'directors' : 'Diretor Desconhecido'}, inplace = True)
df.fillna({'countries' : 'País Desconhecido'}, inplace = True)

In [17]:
df.dropna(subset = ['date_added'], inplace = True)

In [18]:
valor_moda = df['rating'].mode()[0]
df.fillna({'rating': valor_moda}, inplace=True)

In [19]:
df.fillna({'cast' : 'Elenco Desconhecido'},inplace = True)

In [20]:
df.dropna(subset = ['duration'], inplace = True)

In [21]:
df['date_added'] = pd.to_datetime(df['date_added'])

# Organização de dados

In [22]:
df['added_year'] = df['date_added'].dt.year
df['added_month'] = df['date_added'].dt.month_name()
df['added_day'] = df['date_added'].dt.day

# Análise de Filmes e Séries

In [23]:
filmes = df[df['type'] == 'Movie'].copy()
filmes['Duration_min'] = filmes['duration'].str.replace(' min','').astype(float)

In [24]:
series = df[df['type'] == 'TV Show'].copy()
series['duration_Season'] = series['duration'].str.replace(' Season[s]?','', regex = True).astype(int)

In [25]:
import plotly.express as px
import plotly.io as pio

### Distribuição de Conteúdo

In [26]:
distribuicao_conteudo = df['type'].value_counts().to_frame().reset_index()
distribuicao_conteudo.columns = ['Tipo', 'Quantidade']

In [27]:
pio.renderers.default = 'iframe'
grafico_distribuicao_conteudo = px.bar(
    distribuicao_conteudo,
    x = 'Quantidade',
    y = 'Tipo',
    title = 'Distribuição de conteúdos na Netflix',
    template='plotly_white'
)

grafico_distribuicao_conteudo.show()

# Top 10 Países Produtores

In [28]:
top_countries = df[df['countries'] != 'País Desconhecido'].copy()
top_countries = top_countries['countries'].value_counts().reset_index()
top_countries.columns = ['País','Contagem']
top_countries = top_countries.head(10)
top_countries.head(10)

,País,Contagem
0,United States,2809
1,India,972
2,United Kingdom,418
3,Japan,244
4,South Korea,200
5,Canada,181
6,Spain,145
7,France,124
8,Mexico,110
9,Egypt,106


In [29]:
pio.renderers.default = 'iframe'
grafico = px.bar(
    top_countries,
    x = 'País',
    y = 'Contagem',
    title = 'Top 10 países mais produtores de conteúdos na Netflix',
    color = 'País',
    template='plotly_white'
)

grafico.show()

# Conteúdo por Ano

In [30]:
conteudos_por_ano = df.groupby('added_year')['title'].count().reset_index(name='Conteúdos')
conteudos_por_ano.columns = ['Ano','Conteúdos']

In [31]:
grafico_conteudo_por_ano = px.line(
    conteudos_por_ano,
    x = 'Ano',
    y = 'Conteúdos',
    title = 'Quantidade de conteúdos adicionados na netflix anualmente',
    template='plotly_white'
)

grafico_conteudo_por_ano.show()

# Gênero do conteúdo

In [32]:
generos = df.assign(listed_in = df['listed_in'].str.split(', ')).explode('listed_in')

In [33]:
series_e_filmes_lixo = ['International Movies', 'International TV Shows', 'TV Shows', 'Movies']
conteudo_por_genero = generos.groupby('listed_in')['title'].count().reset_index(name='Quantidade')
conteudo_por_genero.columns = ['Gênero','Quantidade']
conteudo_por_genero = conteudo_por_genero[~conteudo_por_genero['Gênero'].isin(series_e_filmes_lixo)]
conteudo_por_genero.sort_values(by = 'Quantidade', ascending = False, inplace = True)
top_conteudos = conteudo_por_genero.head(10)

In [34]:
grafico_generos = px.bar(
    top_conteudos,
    x = 'Gênero',
    y = 'Quantidade',
    title = 'Quantidade de conteúdos por gêneros na Netflix',
    template='plotly_white'
)

grafico_generos.show()

# Análise de partipações em Filmes e Séries

In [36]:
elenco = df.assign(elenco = df['cast'].str.split(', ')).explode('elenco')

In [39]:
integrante = elenco.groupby('elenco')['title'].count().reset_index('elenco')
integrante.columns = ['Ator/Atriz', 'Atuações']
integrante = integrante[integrante['Ator/Atriz'] != 'Elenco Desconhecido']
integrante.sort_values(by = 'Atuações', ascending = False, inplace = True)
integrante = integrante.head(10)
integrante

,Ator/Atriz,Atuações
2830,Anupam Kher,43
30461,Shah Rukh Khan,35
16681,Julie Tejwani,33
32561,Takahiro Sakurai,32
24193,Naseeruddin Shah,32
28946,Rupa Bhimani,31
846,Akshay Kumar,30
25400,Om Puri,30
35847,Yuki Kaji,29
25758,Paresh Rawal,28


In [40]:
grafico_cast = px.bar(
    integrante,
    x = 'Ator/Atriz',
    y = 'Atuações',
    title = 'Top 10 Ator/Atriz com mais partipações em conteúdos na netflix',
    template='plotly_white',
    color = 'Ator/Atriz'
)

grafico_cast.show()